In [ ]:
import pandas as pd
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_extraction import DictVectorizer

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

### what is this ?
1. xgboost module is imported to use the XGBoost model. see "XGBoost.ipynb" where i tried to make a simple regression model
2. hyperopt:It's a library that used some **bayesian methods** to try to find the best set of hyperparameters 
   In terms of hyperopt your model will be just a function that takes some input(combination of hyper parameters) and outputs some values. that function called ***objective function*** 
   1. fmin: Runs the optmization loop to find the best hyperparameter set. (the hp combination that produces the lowest RMSE)
      Used to minimize the objective function(outputs of the model), it depends on the evaluation metric you may need the maximum output.
   2. tpe: The algorithm used to pick the next best parameters to try (TPE = Tree-structured Parzen Estimator)
   3. hp: library that contains a bunch of different methods to define the search space (the ranges of each hyperparameter that we will use).
   4. STATUS_OK: signal that we will send at the end of each run to tell hyperopt that the objective function has run successfully.
   5. Trials: will keep track of the information of each run
3. scope: used to cast values inside the **search space** to integer, float, etc.

summery:
trying different hyper-parameter set generated by the **hp** module, each new set is selected using **tpe** algorithm. the **fmin** is used to run the objective function with different parameter set and selects the parameter set that produces the minimum RMSE. So from its name it tries to minimize the RMSE by running the model again and again with different hyper-parameter set.

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID'
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db") # path to the tracking server database
mlflow.set_experiment("nyc-taxi-experiment") # experiment name, will be              created if not found 

In [ ]:
# DMatrix is an optimized data structure used by XGBoost to:
# Store your features (X_train, X_val) and labels (y_train, y_val)
# Speed up training (it’s faster than using raw NumPy arrays or Pandas)
# Handle things like missing values, sparse data, and caching internally

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

### Hyper-parameter tunning using XGBoost model

In [ ]:

"""
Objective function is responsible for training the model 
with different hyper-parameters and returns RMSE and signal indicates the training has finished successfully

The training works as follows: 
    Start training (up to 1000 rounds)
    After each round:
        Train a new tree
        Evaluate on the validation set
        Check if performance improved
    If it stops improving for 50 rounds in a row → training stops early
    The model automatically keeps the best version it saw during training
"""
def objective(params: dict) -> dict:
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100, # Try training up to 1000 trees (boosting rounds)
            evals=[(valid, 'validation')], # While training, also evaluate the model on this validation set
            # If the validation score doesn't improve for 50 rounds, stop training early
            # this indicates when to do early stopping to avoid overfitting and wasting time
            early_stopping_rounds=20 
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
"""
hp.quniform => from 4 to 100 step 1
"""
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    # this tells the xgboost training algorithm to use RMSE metric in validation after each round
    'objective': 'reg:squarederror',
    # For predictability
    'seed': 42
}

"""
the fn parameter is the objective function that is responsible for running the model
it's expected to take the params as input
and return a dictionary contains two keys "loss" and  "status"
"""
# this call returns a dictionary containing the best parameter set
best_result = fmin(
    fn=objective,
    space=search_space,
    # This tells Hyperopt how to choose the next set of parameters. tpe (Tree-structured Parzen Estimator) is a smart algorithm that learns from past trials to pick better values next time.
    algo=tpe.suggest,
    # number of runs, each run with different Hypter-parameter set
    max_evals=10,
    trials=Trials()
)

In [ ]:
# best hyper parameter combination that makes the lowest loss
best_result

### MLFlow auto logging

I will try to train a new model with best hyper-parameters obtained from the previous step using mlflow.autolog

Autolog works only for specific frameworks one of them is XGBoost

It records the hyper-parameters, metrics, model and lots of information

In [ ]:
best_hps = {
  'learning_rate': 0.06795566766046571,
  'max_depth': 74,
  'min_child_weight': 1.1034760099449035,
  'reg_alpha': 0.08418429054929681,
  'reg_lambda': 0.007240669500118009,
  'objective': 'reg:squarederror',
  'seed': 42
}

In [ ]:
mlflow.xgboost.autolog()

booster = xgb.train(
  params=best_hps,
  dtrain=train,
  num_boost_round=1000,
  evals=[(valid, "validation")],
  early_stopping_rounds=50
)

### Saving and Loading the model to make predictions

#### using mlflow.log_artifact()

Needs the model to be saved previously on the local system, then move it to mlflow artifacts \
To run the model again you need to download it from the artifacts and then load it 

In [ ]:
import pickle
mlflow.xgboost.autolog(disable=True)

with mlflow.start_run():
  mlflow.set_tag("developer", "kamal")
  
  mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
  mlflow.log_param("validation-data-path", "../data/green_tripdata_2021-02.parquet")

  mlflow.log_params(best_hps)

  booster = xgb.train(
    params=best_hps,
    dtrain=train,
    num_boost_round=10,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
  )

  y_valid_predictions = booster.predict(valid)
  rmse = root_mean_squared_error(y_true=y_val, y_pred=y_valid_predictions)

  mlflow.log_metric("rmse", rmse)

  with open("models/booster.bin", "wb") as model_file:
    pickle.dump(booster, model_file)

  mlflow.log_artifact(local_path="models/booster.bin", artifact_path="pickle_models")

In [ ]:
with open("models/booster.bin", "rb") as booster_file:
  loaded_booster = pickle.load(booster_file)

temp = loaded_booster.predict(valid)

In [ ]:
temp[:10]

#### Using mlflow.{FrameWork}.log_model()

This provides a more reliable way to load and deploy the trained model 

In [ ]:
import mlflow.xgboost


mlflow.xgboost.autolog(disable=True)

with mlflow.start_run(run_name="saving_using_mlflow.xgboost.log_model()_with_preprocessor"):
  mlflow.set_tag("developer", "kamal")
  
  mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
  mlflow.log_param("validation-data-path", "../data/green_tripdata_2021-02.parquet")

  mlflow.log_params(best_hps)

  booster = xgb.train(
    params=best_hps,
    dtrain=train,
    num_boost_round=10,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
  )

  y_valid_predictions = booster.predict(valid)
  rmse = root_mean_squared_error(y_true=y_val, y_pred=y_valid_predictions)

  mlflow.log_metric("rmse", rmse)

  with open("models/preprocessor.bin", "wb") as preprocessor_file:
    pickle.dump(dv, preprocessor_file)

  mlflow.log_artifact("models/preprocessor.bin", "preprocessors")


  # artifact_path is deprecated, use name istead
  # Mlflow new versions separts Runs from their models
  # when specifying the "name" parameter then it's the name of the trained model
  mlflow.xgboost.log_model(booster, name="models_mlflow")


### Loading Model from artifacts and make predictions

#### Loading the model as a python function flavour

In [ ]:
logged_model = 'runs:/ab3f2144e5b5471fa60f5ba979ae2a04/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
loaded_model

In [ ]:
loaded_model.predict(X_val)[:3]

#### Loading the model as XGBoost object

In [ ]:
logged_model = 'runs:/ab3f2144e5b5471fa60f5ba979ae2a04/models_mlflow'

loaded_model = mlflow.xgboost.load_model(logged_model)

In [ ]:
loaded_model

In [ ]:
loaded_model.predict(valid)[:3]